In [3]:
!pip3 install scikit-learn

import numpy as np
import json
from sklearn.cluster import KMeans

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/18/ff/919087b9aa5318c3992def31b74e019eee39c57ff6bf140289041df3db86/scikit_learn-1.4.1.post1-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/1e/84/ccd9b08653022b7785b6e3ee070ffb2825841e0dc119be22f0840b2b35cb/threadpoolctl-3.4.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 28.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 20.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [24]:
# Listing out tweets to cluster.
base_path = "/Users/shivamenta/Desktop/embeddings_twitter_data/"
banks = ["all_risk.json"]
tweets = []
clustered_tweets = {}

for bank in banks:
    filepath = base_path + bank
    with open(filepath, "r") as f:
        tweets.extend(json.loads(s) for s in f.readlines())

In [25]:
def k_means_tweets(tweets, num_centroids=3, tweets_per_centroid=3):
    global clustered_tweets
    tweets_per_centroid = max(3, min(tweets_per_centroid, 5))
    
    embeddings = [tweet['embedding'] for tweet in tweets]
    embeddings = np.array(embeddings)
    
    kmeans = KMeans(n_clusters=num_centroids, random_state=0).fit(embeddings)
    tweet_labels = kmeans.labels_
    
    clustered_tweets = {}
    for label, tweet in zip(tweet_labels, tweets):
        if label not in clustered_tweets:
            clustered_tweets[label] = []
        clustered_tweets[label].append(tweet)
    
    for centroid_index, centroid in enumerate(kmeans.cluster_centers_):
        print(f"\nCentroid {centroid_index}:")
        distances = []
        for tweet in clustered_tweets[centroid_index]:
            distance = np.linalg.norm(centroid - np.array(tweet['embedding']))
            distances.append((tweet, distance))
        
        closest_tweets = sorted(distances, key=lambda x: x[1])[:tweets_per_centroid]
        
        for i, (tweet, distance) in enumerate(closest_tweets, start=1):
            print(f"{i}. {tweet['text']} (Distance: {distance:.2f})")

In [27]:
# Running kmeans.
k_means_tweets(tweets, num_centroids=4, tweets_per_centroid=5)


Centroid 0:
1. SunTrust Bank Settles with Feds for Nearly $1 Billion http://t.co/reJxKMfKhi via @DSNewsDaily (Distance: 0.67)
2. Suntrust Reaches $320 Million Settlement http://t.co/lmkH68UBon | by @RobertSiegel1 (Distance: 0.67)
3. SEC Charges Fifth Third Bancorp and Former CFO for Improper Accounting of Loan Losses During Financial Crisis http://t.co/nmjF19DUzA (Distance: 0.68)
4. (#On_Swager_1) SunTrust Mortgage Settles Federal Probe for $320M: SunTrust Mortgage to pay up t... http://t.co/JnFdnE1KZ6 (#On_Swager_1) (Distance: 0.71)
5. SunTrust Bank, U.S. reach settlement in discrimination case: WASHINGTON (Reuters) - SunTrust Bank Inc's mortgage... http://t.co/mKP2YUl2 (Distance: 0.72)

Centroid 1:
1. ok screw Huntington bank and how slow they are #pissedaf (Distance: 0.60)
2. Huntington bank is pissing me off (Distance: 0.61)
3. Huntington Bank was about to have me flip out! (Distance: 0.65)
4. swear i hate huntington bank ! ✋😒 (Distance: 0.65)
5. I liked a @YouTube video from @spl

In [21]:
# Writing these tweets to disk.
all_tweets = []
for label in clustered_tweets:
    for tweet in clustered_tweets[label]:
        if "embedding" in tweet:
            tweet.pop("embedding")
        tweet["centroid_label"] = int(label)
        all_tweets.append(tweet)

with open("/Users/shivamenta/Desktop/labeled_data/risk_embeddings_labeled.json", "w") as f:
    json.dump(all_tweets, f)